In [822]:
import random
import logging
from collections import namedtuple
from operator import attrgetter

In [823]:
def problem(N, seed=None):
    """Creates a list of lists that contains a random amount of numbers between 0 and N-1."""
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [837]:
Individual = namedtuple("Individual", ["genome", "fitness"])

#Global varibles
POPULATION_SIZE = 30
GENERATIONS = 500
N = 30

In [825]:
def unique(solution):
    """Count the number of unique values in the solution """
    unique = set([item for sublist in solution for item in sublist])
    return len(unique)

In [826]:
def fitness(genome): 
    """Optimal fitness=0. A penalty is given for the number of duplicated elements and also for the number of values not covered by the solution"""
    fitness = 0
    
    #count the number of elements in the genome
    size = len([item for sublist in genome for item in sublist])
    
    unique_values = unique(genome)
    
    #the fitness relates to the total number of element in the genome devided by unique elements
    fitness = N*(size/unique_values)

    #number of values not covered by the genome
    values_left = N - unique_values
   
    if (values_left > 0):
        fitness = fitness + values_left*N

    return fitness


In [827]:
def mutation(individual, P):
    """Change one gene (list) inside the solution by randomly selecting one and swapping it for a new one from the original problem"""
    index = random.randrange(len(individual.genome))
    old_gene = individual.genome.pop(index)

    P_index = random.randrange(len(P))
    new_gene = P[P_index]
    
    new_genome = individual.genome + [new_gene]
    new_fitness = fitness(new_genome)
    
    return Individual(new_genome, new_fitness)


In [828]:
"""Create two new childern from two parents by slicing them at a random place(interval)"""
def crossover(first_individual, second_individual):
    minimum_size = min(len(first_individual.genome), len(second_individual.genome))

    interval = random.randrange(minimum_size)


    first_child_genome = second_individual.genome[:interval] + first_individual.genome[interval:]
    second_child_genome = first_individual.genome[:interval] + second_individual.genome[interval:]

    first_child = Individual(first_child_genome, fitness(first_child_genome))
    second_child = Individual(second_child_genome, fitness(second_child_genome))

    return first_child, second_child
    


In [829]:
def select_parent(population, TOURNAMENT_SIZE=5):
    #Tournament pool
    tournament = []

    while len(tournament) != TOURNAMENT_SIZE: 
        random_id = random.randrange(len(population))
        choosen_individual = population[random_id]
        
        tournament.append(choosen_individual)

    tournament.sort(key=attrgetter('fitness'))
    
    return tournament[0]

In [830]:
def generate_individual(P):
    individual_size = random.randrange(1, len(P))
    individual = random.sample(P, individual_size)

    return individual

In [831]:
def generate_population(P, POPULATION_SIZE):
    population = []

    for i in range(POPULATION_SIZE):

        new_individual = generate_individual(P)

        if new_individual not in population: 
            population.append(Individual(new_individual, fitness(new_individual)))

    return population



In [832]:
def create_new_population(population, offspring):
    """Creates a new population that consist of the 15 best parents and the 15 best offspring in regards to thier fitness"""
    new_population = []

    #Sorts the list from fitesst to least fit
    population.sort(key=attrgetter('fitness'))
    offspring.sort(key=attrgetter('fitness'))
    
    best_fitness = min(population[0].fitness, offspring[0].fitness)
    
    new_population = population[:int(POPULATION_SIZE/2)] + offspring[:int(POPULATION_SIZE/2)]
    
    return new_population, best_fitness
    

In [833]:
def create_offspring(population, P):
    """Generates offspring the same size as the population. Each offspring is created by crossover and some will be mutated as well"""
    offspring = []

    for i in range(int(POPULATION_SIZE/2)):
        tournament_parent = select_parent(population)
        
        random_id = random.randrange(POPULATION_SIZE)
        random_parent = population[random_id]

        first_child, second_child = crossover(tournament_parent, random_parent)

        if random.random() < 0.3:
            first_child = mutation(first_child, P)

        if random.random() < 0.3:
            second_child = mutation(second_child, P)

        offspring.append(first_child)
        offspring.append(second_child)

    return offspring

In [834]:
def escape_local_optima(population, P):
    """Creates a new population by mutating all individuals in the population"""
    new_population = []

    for i in range(len(population)):
        
        new_individual = mutation(population[i], P)
        new_population.append(new_individual)

    return new_population

In [835]:
def genetic_algorithm(P, N, generations):
    """Use stady state to generate the best solution."""
    #create inital population
    population = generate_population(P,POPULATION_SIZE)

    
    population.sort(key=attrgetter('fitness'))
    
    #Save the best fintess found so far
    current_best_fitness = population[0].fitness
    
    #Used to keep track of local optima
    counter = 0

    while generations > 0:
        offspring = create_offspring(population, P)
        population, new_best_fitness = create_new_population(population, offspring)
        
        #If the solution has not improved -> potential local optima
        if (new_best_fitness == current_best_fitness):
            counter += 1
        else:
            counter = 0
            current_best_fitnes = new_best_fitness

        #If stuck in a local optima
        if (counter == 5):
            population = escape_local_optima(population, P)
        
        generations -= 1
    
    population.sort(key=attrgetter('fitness'))
    
    return population[0] 

    

In [838]:
P = problem(N, 42)
final_solution = genetic_algorithm(P, N, GENERATIONS)

print(f'LEN: {sum([len(l) for l in final_solution.genome])}')
print(f'LAST: {final_solution}')



LEN: 50
LAST: Individual(genome=[[0, 6, 7, 24, 26], [1, 2, 5, 15, 16, 19, 21, 27, 29], [2, 12, 13, 19, 21, 25], [6, 10, 18, 22, 28, 29], [17, 20, 6, 22], [17, 20, 6, 22], [6, 7, 11, 21, 28], [3, 4, 8, 23, 28], [0, 9, 14, 15, 18, 27]], fitness=50.0)
